In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, glob

TRAIN_IMG = "/content/drive/MyDrive/T1/train/images"
VAL_IMG   = "/content/drive/MyDrive/T1/val/images"

TRAIN_CSV = "/content/train.csv"
VAL_CSV   = "/content/val.csv"
LABEL_TXT = "/content/label_list.txt"

print("TRAIN images:", len(glob.glob(TRAIN_IMG+"/*.jpg")))
print("VAL images:", len(glob.glob(VAL_IMG+"/*.jpg")))
print("TRAIN_CSV exists:", os.path.exists(TRAIN_CSV))
print("VAL_CSV exists:", os.path.exists(VAL_CSV))
print("LABEL_TXT exists:", os.path.exists(LABEL_TXT))

TRAIN images: 4094
VAL images: 50
TRAIN_CSV exists: True
VAL_CSV exists: True
LABEL_TXT exists: True


In [ ]:
!pip -q install timm

In [ ]:
import json, random
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

def seed_everything(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

DEVICE: cuda


In [ ]:
def load_label_list(path):
    labels = [line.strip() for line in open(path, "r") if line.strip()]
    labels_int = [int(x) for x in labels]
    label_to_idx = {lab: i for i, lab in enumerate(labels_int)}  # 1..102 -> 0..101
    idx_to_label = {i: lab for lab, i in label_to_idx.items()}  # 0..101 -> 1..102
    return labels_int, label_to_idx, idx_to_label

labels_int, label_to_idx, idx_to_label = load_label_list(LABEL_TXT)
NUM_CLASSES = len(labels_int)

print("NUM_CLASSES:", NUM_CLASSES)

NUM_CLASSES: 102


In [ ]:
class FlowerDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform, has_labels=True):
        self.df = pd.read_csv(csv_path, dtype=str)
        self.img_dir = img_dir
        self.transform = transform
        self.has_labels = has_labels

        if "filename" not in self.df.columns:
            raise ValueError(f"{csv_path} missing 'filename' column")
        if has_labels and "label" not in self.df.columns:
            raise ValueError(f"{csv_path} missing 'label' column")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.img_dir, row["filename"])
        img = Image.open(path).convert("RGB")
        x = self.transform(img)

        if self.has_labels:
            y = int(row["label"])          # 1..102
            y_idx = label_to_idx[y]        # 0..101
            return x, y_idx
        return x

In [ ]:
IMG_SIZE = 384
BATCH_SIZE = 32     # bigger images -> smaller batch
NUM_WORKERS = 2

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.ColorJitter(0.25,0.25,0.25,0.1)], p=0.5),
    transforms.RandomRotation(12),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

val_tfms = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.14)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

train_ds = FlowerDataset(TRAIN_CSV, TRAIN_IMG, train_tfms, has_labels=True)
val_ds   = FlowerDataset(VAL_CSV,   VAL_IMG,   val_tfms,   has_labels=True)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=True)

print("train batches:", len(train_loader), "val batches:", len(val_loader))

train batches: 128 val batches: 2


In [ ]:
MODEL_NAME = "convnext_tiny"
EPOCHS = 20
LR = 2e-4
WD = 1e-4
LABEL_SMOOTH = 0.1

model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=NUM_CLASSES).to(DEVICE)

criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTH)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [ ]:
def macro_f1_present_only(conf):
    eps = 1e-9
    support = conf.sum(1)          # how many true samples per class
    present = support > 0

    tp = conf.diag()
    fp = conf.sum(0) - tp
    fn = conf.sum(1) - tp

    f1 = (2*tp) / (2*tp + fp + fn + eps)
    if present.any():
        return f1[present].mean().item(), int(present.sum().item())
    return 0.0, 0

@torch.no_grad()
def evaluate(model):
    model.eval()
    conf = torch.zeros((NUM_CLASSES, NUM_CLASSES), dtype=torch.float32)
    correct, total = 0, 0
    total_loss, n = 0.0, 0

    for x, y in val_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        logits = model(x)
        loss = criterion(logits, y)
        total_loss += loss.item() * x.size(0)
        n += x.size(0)

        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.numel()

        for t, p in zip(y.cpu(), pred.cpu()):
            conf[t, p] += 1

    acc = correct / max(total, 1)
    f1p, k = macro_f1_present_only(conf)
    return total_loss / max(n, 1), acc, f1p, k

In [ ]:
MODEL_OUT = "/content/drive/MyDrive/T1/model"
os.makedirs(MODEL_OUT, exist_ok=True)

best_f1p = -1.0
best_path = os.path.join(MODEL_OUT, "weights.pt")

for epoch in range(1, EPOCHS+1):
    model.train()
    running, seen = 0.0, 0

    for x, y in train_loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad(set_to_none=True)

        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running += loss.item() * x.size(0)
        seen += x.size(0)

    scheduler.step()

    val_loss, val_acc, val_f1p, k = evaluate(model)
    print(f"Epoch {epoch:02d}/{EPOCHS} | train_loss={running/seen:.4f} | val_loss={val_loss:.4f} | val_acc={val_acc:.3f} | macroF1_present={val_f1p:.3f} (classes={k})")

    if val_f1p > best_f1p:
        best_f1p = val_f1p
        torch.save(model.state_dict(), best_path)

# Save meta for predict.py
meta = {
    "model_name": MODEL_NAME,
    "img_size": IMG_SIZE,
    "num_classes": NUM_CLASSES,
    "idx_to_label": idx_to_label,
}
with open(os.path.join(MODEL_OUT, "meta.json"), "w") as f:
    json.dump(meta, f)

print("\nBest macroF1_present:", best_f1p)
print("Saved model to:", MODEL_OUT)

Epoch 01/20 | train_loss=4.6402 | val_loss=4.5396 | val_acc=0.060 | macroF1_present=0.003 (classes=37)
Epoch 02/20 | train_loss=4.6063 | val_loss=4.5432 | val_acc=0.040 | macroF1_present=0.002 (classes=37)
Epoch 03/20 | train_loss=4.5577 | val_loss=4.3199 | val_acc=0.120 | macroF1_present=0.013 (classes=37)
Epoch 04/20 | train_loss=4.4703 | val_loss=4.1559 | val_acc=0.080 | macroF1_present=0.035 (classes=37)
Epoch 05/20 | train_loss=4.4228 | val_loss=4.0661 | val_acc=0.060 | macroF1_present=0.027 (classes=37)
Epoch 06/20 | train_loss=4.3512 | val_loss=4.0203 | val_acc=0.100 | macroF1_present=0.057 (classes=37)
Epoch 07/20 | train_loss=4.2865 | val_loss=3.8811 | val_acc=0.120 | macroF1_present=0.058 (classes=37)
Epoch 08/20 | train_loss=4.2096 | val_loss=3.7378 | val_acc=0.140 | macroF1_present=0.060 (classes=37)
Epoch 09/20 | train_loss=4.1073 | val_loss=3.6164 | val_acc=0.200 | macroF1_present=0.116 (classes=37)
Epoch 10/20 | train_loss=3.9654 | val_loss=3.2937 | val_acc=0.300 | macro

In [ ]:
model.load_state_dict(torch.load(best_path, map_location=DEVICE))
model.eval()

x, y = next(iter(val_loader))
with torch.no_grad():
    logits = model(x.to(DEVICE))
pred_idx = logits.argmax(dim=1).cpu().numpy()

pred_labels = [idx_to_label[int(i)] for i in pred_idx[:10]]
true_labels = [idx_to_label[int(i)] for i in y[:10].numpy()]

print("pred:", pred_labels)
print("true:", true_labels)

pred: [55, 96, 62, 96, 9, 94, 83, 1, 88, 95]
true: [45, 1, 96, 78, 9, 94, 48, 1, 88, 32]
